#### Simple GenAI app using Langchain and OpenAI

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
#Data ingestion from websites
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.langchain.com/oss/python/langchain/quickstart")
docs = loader.load()  # --- IGNORE ---

In [ ]:
#split the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs) 

In [ ]:
documents

In [ ]:
#embded the documents
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
#store the embeddings in a vector database
from langchain.vectorstores import FAISS
vectorstordb=FAISS.from_documents(documents,embeddings)

In [ ]:
vectorstordb

In [ ]:
#search from vector database
query="Tools can depend on runtime context and also interact"
results=vectorstordb.similarity_search(query)
results[0].page_content

In [ ]:
query="Tools can depend on runtime context and also interact"
results=vectorstordb.similarity_search_with_score(query)
results

In [ ]:
#Retrieval chain, document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

""")
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Tools can depend on runtime context and also interact",
    "context": [Document(page_content="Tools let a model interact with external systems by calling functions you define. Tools can depend on runtime context and also interact with agent memory")],
 
})

##### We need the documents to come from the retrievers

In [ ]:
## input --> retriver --> vectorDB

retriever = vectorstordb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever, document_chain)

In [ ]:
retrieval_chain

In [ ]:
# Get the response from the retrieval chain
response = retrieval_chain.invoke({"input":"Tools can depend on runtime context and also interact"})

In [ ]:
response['answer']

In [ ]:
response['context']